In [31]:
import requests
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from config import weather_api_key

In [33]:
url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?"
units = "imperial"

In [34]:
uber_csv = pd.read_csv("Resources/uber.csv")

uber_df = pd.DataFrame(uber_csv)

uber_df.head()



Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1  2009-07-17 20:04:56 UTC        -73.994355        40.728225   
2  2009-08-24 21:45:00 UTC        -74.005043        40.740770   
3  2009-06-26 08:22:21 UTC        -73.976124        40.790844   
4  2014-08-28 17:47:00 UTC        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  
0         -73.999512         40.723217                1  
1         -73.994710         40.750325                1  
2         -73.962565         40.772647                1  
3         -73.965316         40.803349                3  
4         -73.973082         40.761247                5

In [35]:
uber_df.rename(columns = {'Unnamed: 0':'ID', 'key':'N/A', 'fare_amount':'Charge', "pickup_datetime":"Pickup Time",  "pickup_longitude": "Pickup Lon", "pickup_latitude":"Pickup Lat",  "dropoff_longitude": "Dropoff Lon", "dropoff_latitude": "Dropoff Lat", "passenger_count": "Passenger Count"}, inplace = True)

uber_df.head()



ID                            N/A  Charge              Pickup Time  \
0  24238194    2015-05-07 19:52:06.0000003     7.5  2015-05-07 19:52:06 UTC   
1  27835199    2009-07-17 20:04:56.0000002     7.7  2009-07-17 20:04:56 UTC   
2  44984355   2009-08-24 21:45:00.00000061    12.9  2009-08-24 21:45:00 UTC   
3  25894730    2009-06-26 08:22:21.0000001     5.3  2009-06-26 08:22:21 UTC   
4  17610152  2014-08-28 17:47:00.000000188    16.0  2014-08-28 17:47:00 UTC   

   Pickup Lon  Pickup Lat  Dropoff Lon  Dropoff Lat  Passenger Count  
0  -73.999817   40.738354   -73.999512    40.723217                1  
1  -73.994355   40.728225   -73.994710    40.750325                1  
2  -74.005043   40.740770   -73.962565    40.772647                1  
3  -73.976124   40.790844   -73.965316    40.803349                3  
4  -73.925023   40.744085   -73.973082    40.761247                5

In [36]:
uber_df3 = uber_df.sample(n=5)
uber_df3

ID                            N/A  Charge  \
186008  33433171    2010-04-27 07:07:56.0000001     6.9   
187118  17796776    2014-07-12 17:52:42.0000002     2.5   
22916   54072714   2013-03-30 08:58:00.00000074     9.0   
126453  14046100  2009-12-11 19:21:00.000000160    10.9   
49566   27388858    2010-03-12 07:23:18.0000002     9.3   

                    Pickup Time  Pickup Lon  Pickup Lat  Dropoff Lon  \
186008  2010-04-27 07:07:56 UTC  -73.950577   40.775355   -73.968273   
187118  2014-07-12 17:52:42 UTC  -73.980650   40.765137   -73.980650   
22916   2013-03-30 08:58:00 UTC  -73.945958   40.804202   -73.967150   
126453  2009-12-11 19:21:00 UTC  -74.005518   40.740853   -74.011852   
49566   2010-03-12 07:23:18 UTC  -73.969618   40.798014   -73.983051   

        Dropoff Lat  Passenger Count  
186008    40.761989                1  
187118    40.765137                1  
22916     40.772317                1  
126453    40.704435                1  
49566     40.762598                1

In [37]:
uber_df3['Date Formatted'] = (pd.to_datetime(uber_df3['Pickup Time'], yearfirst=True, utc=True)) 
x = (uber_df3["Date Formatted"] - pd.Timestamp("1970-01-01", tz="UTC")) // pd.Timedelta('1s') 
uber_df3


ID                            N/A  Charge  \
186008  33433171    2010-04-27 07:07:56.0000001     6.9   
187118  17796776    2014-07-12 17:52:42.0000002     2.5   
22916   54072714   2013-03-30 08:58:00.00000074     9.0   
126453  14046100  2009-12-11 19:21:00.000000160    10.9   
49566   27388858    2010-03-12 07:23:18.0000002     9.3   

                    Pickup Time  Pickup Lon  Pickup Lat  Dropoff Lon  \
186008  2010-04-27 07:07:56 UTC  -73.950577   40.775355   -73.968273   
187118  2014-07-12 17:52:42 UTC  -73.980650   40.765137   -73.980650   
22916   2013-03-30 08:58:00 UTC  -73.945958   40.804202   -73.967150   
126453  2009-12-11 19:21:00 UTC  -74.005518   40.740853   -74.011852   
49566   2010-03-12 07:23:18 UTC  -73.969618   40.798014   -73.983051   

        Dropoff Lat  Passenger Count            Date Formatted  
186008    40.761989                1 2010-04-27 07:07:56+00:00  
187118    40.765137                1 2014-07-12 17:52:42+00:00  
22916     40.772317                1 2013-03-30 08:58:00+00:00  
126453    40.704435                1 2009-12-11 19:21:00+00:00  
49566     40.762598                1 2010-03-12 07:23:18+00:00

In [38]:
uber_df3['Date Formatted Final'] = (uber_df3["Date Formatted"] - pd.Timestamp("1970-01-01", tz="UTC")) // pd.Timedelta('1s')
uber_df3



ID                            N/A  Charge  \
186008  33433171    2010-04-27 07:07:56.0000001     6.9   
187118  17796776    2014-07-12 17:52:42.0000002     2.5   
22916   54072714   2013-03-30 08:58:00.00000074     9.0   
126453  14046100  2009-12-11 19:21:00.000000160    10.9   
49566   27388858    2010-03-12 07:23:18.0000002     9.3   

                    Pickup Time  Pickup Lon  Pickup Lat  Dropoff Lon  \
186008  2010-04-27 07:07:56 UTC  -73.950577   40.775355   -73.968273   
187118  2014-07-12 17:52:42 UTC  -73.980650   40.765137   -73.980650   
22916   2013-03-30 08:58:00 UTC  -73.945958   40.804202   -73.967150   
126453  2009-12-11 19:21:00 UTC  -74.005518   40.740853   -74.011852   
49566   2010-03-12 07:23:18 UTC  -73.969618   40.798014   -73.983051   

        Dropoff Lat  Passenger Count            Date Formatted  \
186008    40.761989                1 2010-04-27 07:07:56+00:00   
187118    40.765137                1 2014-07-12 17:52:42+00:00   
22916     40.772317                1 2013-03-30 08:58:00+00:00   
126453    40.704435                1 2009-12-11 19:21:00+00:00   
49566     40.762598                1 2010-03-12 07:23:18+00:00   

        Date Formatted Final  
186008            1272352076  
187118            1405187562  
22916             1364633880  
126453            1260559260  
49566             1268378598

In [45]:
clean_df = uber_df3[["ID", "Charge", "Pickup Lon", "Pickup Lat", "Dropoff Lon", "Dropoff Lat", "Passenger Count", "Date Formatted Final" ]]


clean_df

ID  Charge  Pickup Lon  Pickup Lat  Dropoff Lon  Dropoff Lat  \
186008  33433171     6.9  -73.950577   40.775355   -73.968273    40.761989   
187118  17796776     2.5  -73.980650   40.765137   -73.980650    40.765137   
22916   54072714     9.0  -73.945958   40.804202   -73.967150    40.772317   
126453  14046100    10.9  -74.005518   40.740853   -74.011852    40.704435   
49566   27388858     9.3  -73.969618   40.798014   -73.983051    40.762598   

        Passenger Count  Date Formatted Final  
186008                1            1272352076  
187118                1            1405187562  
22916                 1            1364633880  
126453                1            1260559260  
49566                 1            1268378598

In [ ]:
temp = []
hum = []
desc = []

for index, row in clean_df.iterrows():

# for row in uber:
    lat = row['Pickup Lat']
    lon = row['Pickup Lon']
    date = row['Date Formatted Final'].astype(int)
    query_url = f"{url}lat={lat}&lon={lon}&dt={date}&appid={api_key}"
    response = requests.get(query_url)
    response_json = response.json()
    temp.append(response[])

https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.789082&lon=-73.948758&dt=1334219940&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.789082
1334219940
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.759057&lon=-73.96809&dt=1327868160&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.759057
1327868160
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.756111&lon=-73.99246&dt=1416263727&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.756111
1416263727
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.770288&lon=-73.962631&dt=1371212416&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.770288
1371212416
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.754755&lon=-73.971152&dt=1250462340&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.754755
1250462340


In [ ]:
response = requests.get(query_url)
response_json = response.json()

print(response_json)

{'lat': 40.7548, 'lon': -73.9712, 'timezone': 'America/New_York', 'timezone_offset': -14400, 'data': [{'dt': 1250462340, 'sunrise': 1250417252, 'sunset': 1250466775, 'temp': 301.59, 'feels_like': 303.67, 'pressure': 1020, 'humidity': 63, 'dew_point': 293.87, 'clouds': 0, 'visibility': 10000, 'wind_speed': 5.1, 'wind_deg': 160, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}]}


In [46]:
# Create dataframe
weather_df = {'Temperature':temp,
              'Humidity':hum,
              'Description':desc

}

In [47]:
row['Temperatur'] = response_json['data']['temp']
row['Humidity'] = response_json['data']['humidity']
row['Desccription'] = response_json['data']['weather']['description']

TypeError: list indices must be integers or slices, not str